# 2 Deploying a Fast AI model to Hugging Face
> Steps to get the hugging face gradio application to work

- toc: false 
- badges: true
- comments: true
- categories: [sound, hugging face, fastai]
- image: images/Hugging-Face.png

## Know before getting started

In [12]:
#collapse-hide

launch_options = {
    "enable_queue": True,
    "share": False,
    #"cache_examples": True,
}

demo.launch(**launch_options)


Running on local URL:  http://127.0.0.1:7861/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f671108ce50>, 'http://127.0.0.1:7861/', None)

## Hugging Face 

The final end point once you assemble all this work through the logs (to resolve errors) is [here](https://huggingface.co/spaces/gputrain/UrbanSounds8K)

![](images/logs.PNG)